In [1]:
import pulp
import numpy as np
np.set_printoptions(formatter={'float': lambda x: "{0:0.5f}".format(x)})
from copy import copy

In [2]:
games = np.load("games.npz")
for game_name in sorted(games.keys()):
    G = games[game_name]
    P = G.shape[-1]
    A = G.shape[:-1]
    print(game_name + ":", P, "players,", A, "actions")

bach_or_stravinsky: 2 players, (2, 2) actions
chicken: 2 players, (2, 2) actions
five_by_five: 2 players, (5, 5) actions
four_players: 4 players, (2, 2, 2, 2) actions
hawk_dove: 2 players, (2, 2) actions
matching_pennies: 2 players, (2, 2) actions
penalty_kick: 2 players, (2, 2) actions
pk_3_actions: 2 players, (3, 3) actions
prisoners_dilemma: 2 players, (2, 2) actions
robot_escape: 2 players, (2, 2) actions
rock_paper_scissors: 2 players, (3, 3) actions
rps_both_hate_ties: 2 players, (3, 3) actions
rps_p1_likes_rock: 2 players, (3, 3) actions
rps_p2_dislikes_ties: 2 players, (3, 3) actions
two_by_three_by_four: 3 players, (2, 3, 4) actions


In [3]:
chicken = games["chicken"]
for p in range(chicken.shape[-1]):
    print(chicken[:,:,p])

[[-100    1]
 [  -1    0]]
[[-100   -1]
 [   1    0]]


The following linear program finds the correlated equilibrium of chicken that's __worst__ for player 1.

In [4]:
CE_lp = pulp.LpProblem("chicken_CE", pulp.LpMinimize)
p_GG = pulp.LpVariable("Pr_GG", 0, 1)
p_GS = pulp.LpVariable("Pr_GS", 0, 1)
p_SG = pulp.LpVariable("Pr_SG", 0, 1)
p_SS = pulp.LpVariable("Pr_SS", 0, 1)

CE_lp += p_GG + p_GS + p_SG + p_SS == 1 # probabilities sum to 1

# add constraints for: each player, each action they might be told to play, and each deviation to a different action
p = 0; a = 0; d = 1
CE_lp += p_GG*chicken[a,0,p] + p_GS*chicken[a,1,p] >= p_GG*chicken[d,0,p] + p_GS*chicken[d,1,p] # G better than S for P1 when told to play G
a = 1; d = 0
CE_lp += p_SG*chicken[a,0,p] + p_SS*chicken[a,1,p] >= p_SG*chicken[d,0,p] + p_SS*chicken[d,1,p] # S better than G for P1 when told to play S
p = 1; a = 0; d = 1
CE_lp += p_GG*chicken[0,a,p] + p_SG*chicken[1,a,p] >= p_GG*chicken[0,d,p] + p_GS*chicken[1,d,p] # G better than S for P2 when told to play G
a = 1; d = 0
CE_lp += p_SS*chicken[a,0,p] + p_GS*chicken[a,1,p] >= p_SS*chicken[d,0,p] + p_GS*chicken[d,1,p] # S better than G for P2 when told to play S

distr_vars = np.array([[p_GG, p_GS], [p_SG, p_SS]])
u1_D = pulp.lpSum(distr_vars * chicken[:,:,0])
u2_D = pulp.lpSum(distr_vars * chicken[:,:,1])
CE_lp += u1_D

print(CE_lp)
CE_lp.solve(solver=pulp.PULP_CBC_CMD(msg=0))
CE = np.zeros([2,2])
for i in np.ndindex(CE.shape):
    CE[i] = distr_vars[i].varValue

print("Found correlated equilibrium:")
print(CE)

chicken_CE:
MINIMIZE
-100*Pr_GG + 1*Pr_GS + -1*Pr_SG + 0
SUBJECT TO
_C1: Pr_GG + Pr_GS + Pr_SG + Pr_SS = 1

_C2: - 99 Pr_GG + Pr_GS >= 0

_C3: 99 Pr_SG - Pr_SS >= 0

_C4: - 99 Pr_GG + Pr_SG >= 0

_C5: Pr_GS + 101 Pr_SS >= 0

VARIABLES
Pr_GG <= 1 Continuous
Pr_GS <= 1 Continuous
Pr_SG <= 1 Continuous
Pr_SS <= 1 Continuous

Found correlated equilibrium:
[[0.00000 0.00000]
 [1.00000 0.00000]]


The following linear program finds the coarse correlated equilibrium of chicken that's __worst__ for player 2.

Note that the version in the in-class activity had a couple of typos!

In [5]:
CCE_lp1 = pulp.LpProblem("chicken_CCE", pulp.LpMinimize)

CCE_lp1 += p_GG + p_GS + p_SG + p_SS == 1 # probabilities sum to 1

# add constraints for: each player and each action they might deviate to
p = 0; a = 0
CCE_lp1 += u1_D >= (p_GG+p_SG)*chicken[a,0,p] + (p_GS+p_SS)*chicken[a,1,p] # Distr better than G for P1
a = 1
CCE_lp1 += u1_D >= (p_GG+p_SG)*chicken[a,0,p] + (p_GS+p_SS)*chicken[a,1,p] # Distr better than S for P1
p = 1; a = 0
CCE_lp1 += u2_D >= (p_GG+p_GS)*chicken[0,a,p] + (p_SG+p_SS)*chicken[1,a,p] # Distr better than G for P2
a = 1
CCE_lp1 += u2_D >= (p_GG+p_GS)*chicken[0,a,p] + (p_SG+p_SS)*chicken[1,a,p] # Distr better than G for P2

CCE_lp1 += u2_D

print(CCE_lp1)
CCE_lp1.solve(solver=pulp.PULP_CBC_CMD(msg=0))
CCE = np.zeros([2,2])
for i in np.ndindex(CCE.shape):
    CCE[i] = distr_vars[i].varValue

print("Found correlated equilibrium:")
print(CCE)

chicken_CCE:
MINIMIZE
-100*Pr_GG + -1*Pr_GS + 1*Pr_SG + 0
SUBJECT TO
_C1: Pr_GG + Pr_GS + Pr_SG + Pr_SS = 1

_C2: 0 Pr_GG + 0 Pr_GS + 99 Pr_SG - Pr_SS >= 0

_C3: - 99 Pr_GG + Pr_GS + 0 Pr_SG >= 0

_C4: 0 Pr_GG + 99 Pr_GS + 0 Pr_SG - Pr_SS >= 0

_C5: - 99 Pr_GG + 0 Pr_GS + Pr_SG >= 0

VARIABLES
Pr_GG <= 1 Continuous
Pr_GS <= 1 Continuous
Pr_SG <= 1 Continuous
Pr_SS <= 1 Continuous

Found correlated equilibrium:
[[0.00000 1.00000]
 [0.00000 0.00000]]


The following linear program for finding a coarse correlated equilibrium in chicken should be equivalent to the one above, but demonstrates a few techniques that might be helpful for generalizing it.

In [30]:
CCE_lp2 = pulp.LpProblem("chicken_CCE", pulp.LpMinimize)

# could re-use the variables from above, but this demonstrates another approach
distr_vars = np.empty(chicken.shape[:-1], dtype=pulp.LpVariable)
for outcome_tuple in np.ndindex(chicken.shape[:-1]):
    distr_vars[outcome_tuple] = pulp.LpVariable("q_" + "".join(str(a) for a in outcome_tuple), 0, 1)

CCE_lp2 += pulp.lpSum(distr_vars) == 1 # probabilities sum to 1

u1_D = pulp.lpSum(distr_vars * chicken[:,:,0])
u2_D = pulp.lpSum(distr_vars * chicken[:,:,1])

CCE_lp2 += u2_D # minimize p2's utility


p1_beliefs = [pulp.lpSum(distr_vars[:,p2_act]) for p2_act in range(2)]
p1_opp_outcomes = [list(outcome_tuple) for outcome_tuple in np.ndindex(chicken.shape[1:2])]
for p1_act in range(2):
    indices = [tuple([p1_act] + o + [0]) for o in p1_opp_outcomes]
    CCE_lp2 += u1_D >= pulp.lpSum(prob * chicken[out] for prob,out in zip(p1_beliefs,indices))

p2_beliefs = [pulp.lpSum(distr_vars[p1_act,:]) for p1_act in range(2)]
p2_opp_outcomes = [list(outcome_tuple) for outcome_tuple in np.ndindex(chicken.shape[0:1])]
for p2_act in range(2):
    indices = [tuple(o + [p2_act, 1]) for o in p2_opp_outcomes]
    CCE_lp2 += u2_D >= pulp.lpSum(prob * chicken[out] for prob,out in zip(p2_beliefs,indices))

print(CCE_lp2)
CCE_lp2.solve(solver=pulp.PULP_CBC_CMD(msg=0))
CCE = np.zeros([2,2])
for i in np.ndindex(CCE.shape):
    CCE[i] = distr_vars[i].varValue

print("Found coarse correlated equilibrium:")
print(CCE)

chicken_CCE:
MINIMIZE
-100*q_00 + -1*q_01 + 1*q_10 + 0
SUBJECT TO
_C1: q_00 + q_01 + q_10 + q_11 = 1

_C2: 0 q_00 + 0 q_01 + 99 q_10 - q_11 >= 0

_C3: - 99 q_00 + q_01 + 0 q_10 >= 0

_C4: 0 q_00 + 99 q_01 + 0 q_10 - q_11 >= 0

_C5: - 99 q_00 + 0 q_01 + q_10 >= 0

VARIABLES
q_00 <= 1 Continuous
q_01 <= 1 Continuous
q_10 <= 1 Continuous
q_11 <= 1 Continuous

Found coarse correlated equilibrium:
[[0.00000 1.00000]
 [0.00000 0.00000]]


Generalize the linear programming approaches illustrated above to compute a correlated or coarse corrleated equilibrium in an arbitrary normal-form game. In each case, you should find the equilibrium that __maximizes total utility__.

In [8]:
def correlated_equilibrium(game):
    
    num_players = game.shape[-1]
    CE_lp = pulp.LpProblem("CE_LP", pulp.LpMaximize)
    vars_arr = np.array(np.empty(game.shape[:-1], dtype = object))
    
    for index in np.ndindex(vars_arr.shape):
        var_name = "q"+"".join(map(str, index))
        vars_arr[index] = pulp.LpVariable(var_name, 0, 1)
        
    CE_lp += vars_arr.sum() == 1
    
    for p in range(num_players):
        for a in range(game.shape[p]):
            var_ind = [slice(None)]* num_players
            var_ind[p] = a
            pay_ind = [slice(None)]* (num_players + 1)
            pay_ind[-1] = p
            for d in range(game.shape[p]):
                if d != a:
                    pay_ind[p] = a
                    deviate_ind = copy(pay_ind)
                    deviate_ind[p] = d
                    
                    CE_lp += (vars_arr[tuple(var_ind)] * game[tuple(pay_ind)]).sum() >= (vars_arr[tuple(var_ind)] * game[tuple(deviate_ind)]).sum()
    
    objective = 0
    for p in range(num_players):
        ind = [slice(None)] * (num_players + 1)
        ind[-1] = p
        objective += pulp.lpSum(vars_arr * game[ind])
    CE_lp += objective

    CE_lp.solve(solver=pulp.PULP_CBC_CMD(msg=0))
    CE = np.zeros(vars_arr.shape)
    for i in np.ndindex(CE.shape):
        CE[i] = vars_arr[i].varValue
        
    return CE

In [10]:
def coarse_correlated_equilibrium(game):
    num_players = game.shape[-1]
    CCE_lp = pulp.LpProblem("CCE_LP", pulp.LpMinimize)
    vars_arr = np.array(np.empty(game.shape[:-1], dtype = object))
    for index in np.ndindex(vars_arr.shape):
        var_name = "q"+"".join(map(str, index))
        vars_arr[index] = pulp.LpVariable(var_name, 0, 1)
        
    
    CCE_lp += vars_arr.sum() == 1
    
    for p in range(num_players):
        ind = [slice(None)] * (num_players + 1)
        ind[-1] = p
        cur_play_util = pulp.lpSum(vars_arr * game[tuple(ind)])
        ind = [slice(None)]* num_players
        for p_act in range(game.shape[p]):
            ind[p] = p_act
            belief = [pulp.lpSum(vars_arr[tuple(ind)])]
        opp_outcomes = [list(outcome_tuple) for outcome_tuple in np.ndindex(game.shape[p])]
        for p_act in range(game.shape[p]):
            indices = [tuple([p_act] + o + [0]) for o in opp_outcomes]
            CCE_lp += cur_play_util >= pulp.lpSum(prob * game[tuple(out)] for prob,out in zip(belief,indices))
    
    objective = 0        
    for p in range(num_players):
        ind = [slice(None)] * (num_players + 1)
        ind[-1] = p
        objective += pulp.lpSum(vars_arr * game[tuple(ind)])
        
    CCE_lp += objective

    CCE_lp.solve(solver=pulp.PULP_CBC_CMD(msg=0))
    
    CCE = np.zeros(vars_arr.shape)
    for i in np.ndindex(CCE.shape):
        CCE[i] = vars_arr[i].varValue

    return CCE
